In [1]:
from bs4 import BeautifulSoup
import urllib.request
import csv

<h3>1. Data Scraping<h3/>

In [2]:
urlpage =  'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [3]:
page = urllib.request.urlopen(urlpage)
soup = BeautifulSoup(page, 'html.parser')

In [4]:
table = soup.find('table', attrs={'class': 'wikitable sortable'})
results = table.find_all('tr')
print('Number of results', len(results))

Number of results 181


In [5]:
rows = []
rows.append(['Postal Code', 'Borough', 'Neighbor'])
print(rows)

[['Postal Code', 'Borough', 'Neighbor']]


In [75]:
for result in results:
    # find all columns per result
    data = result.find_all('td')
    # check that columns have data 
    if len(data) == 0: 
        continue
    # write columns to variables
    pos = data[0].get_text()
    bor = data[1].get_text()
    neigh = data[2].get_text()
    rows.append([pos, bor, neigh])

<h3> 2. Write Data to csv File<h3/>

In [162]:
with open('TNeigh.csv','w', encoding="utf-8") as f_output:
    csv_output = csv.writer(f_output)
    csv_output.writerows(rows)

In [163]:
import pandas as pd
f1 = pd.read_csv('TNeigh.csv')

In [164]:
f1.head()

,Postal Code,Borough,Neighbor
0,M1A\r\n,Not assigned\r\n,Not assigned\r\n
1,M2A\r\n,Not assigned\r\n,Not assigned\r\n
2,M3A\r\n,North York\r\n,Parkwoods\r\n
3,M4A\r\n,North York\r\n,Victoria Village\r\n
4,M5A\r\n,Downtown Toronto\r\n,"Regent Park, Harbourfront\r\n"


In [165]:
f1.shape

(360, 3)

Necessary data cleanups

In [166]:
import re
for i in f1:
    f1[i] = f1[i].astype(str).str.rstrip()

In [167]:
f1.head()

,Postal Code,Borough,Neighbor
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [168]:
indexNames = f1[f1['Borough'] == 'Not assigned'].index

In [169]:
f1.drop(indexNames, inplace = True)

In [170]:
f1.shape

(206, 3)

In [173]:
f1.head()

,Postal Code,Borough,Neighbor
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [174]:
indexNames2 = f1[f1['Neighbor'] == 'Not assigned'].index
indexNames2

Int64Index([], dtype='int64')

<h3> 3. Merging Data<h3/>

In [175]:
f1 = f1.groupby(['Postal Code', 'Borough'])['Neighbor'].apply(','.join).reset_index()

In [176]:
f1.shape

(103, 3)

In [179]:
f1.head()

,Postal Code,Borough,Neighbor
0,M1B,Scarborough,"Malvern, Rouge,Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek,Rouge H..."
2,M1E,Scarborough,"Guildwood, Morningside, West Hill,Guildwood, M..."
3,M1G,Scarborough,"Woburn,Woburn"
4,M1H,Scarborough,"Cedarbrae,Cedarbrae"
